In [1]:
%cd /app

/app


In [2]:
import argparse
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import torch
torch.multiprocessing.set_start_method('spawn')

import jax
from lob.encoding import Vocab, Message_Tokenizer

from lob import inference_no_errcorr as inference
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint

from lob import inference_no_errcorr as inference
import lob.encoding as encoding
import preproc as preproc

import jax.numpy as jnp
import numpy as np

from pathlib import Path
import os

import pandas as pd
import plotly.graph_objs as go
import yaml

from filtration_utils import summary_table, build_zero_padded_series, plot_midprice_series_with_insertions, prepare_volatility_filtered_series, plot_midprice_series_with_mean_std

2025-07-31 23:25:05.756022: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.6 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2025-07-31 23:25:08.862269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# experiment_name = 'exp_99_20250708_173043'
experiment_name = 'exp_40_20250731_232348' #''   exp_28_20250724_192236


CONFIG_PATH = f"/app/data_saved/{experiment_name}/used_config.yaml"

# Load YAML config
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Extract values
num_insertions      = config["num_insertions"]
num_coolings        = config["num_coolings"]
midprice_step_size  = config["midprice_step_size"]
hist_msgs           = config["n_messages"]
n_gen_msgs          = config["n_gen_msgs"]
Direction           = config["DIRECTION_i"]

In [4]:
print(f"Aggressive {'buy' if Direction == 0 else 'sell'}\n")
print(f'num_insertions: {num_insertions}')
print(f'num_coolings: {num_coolings}')
print(f'midprice_step_size: {midprice_step_size}')
print(f'hist_msgs: {hist_msgs}')
print(f'n_gen_msgs: {n_gen_msgs}')

Aggressive buy

num_insertions: 10
num_coolings: 30
midprice_step_size: 1
hist_msgs: 500
n_gen_msgs: 50


In [5]:
500+(10+30)*51

2540

In [6]:
hist_steps = hist_msgs // midprice_step_size       # 500
gen_steps = n_gen_msgs // midprice_step_size     # 50
gen_block = gen_steps + 1                        # 51

merged = summary_table(experiment_name)
x, all_series = build_zero_padded_series(hist_msgs, n_gen_msgs, midprice_step_size, merged)

print(merged)

     id                                        merged_data
0  1086  [881800, 881800, 881800, 881800, 881800, 88190...
1  2640  [868600, 868600, 868600, 868600, 868600, 86860...
2  2674  [873400, 873400, 873400, 873400, 873400, 87340...
3  4030  [890900, 890900, 890900, 890900, 890900, 89090...


In [7]:
fig = plot_midprice_series_with_insertions(
    merged,
    all_series,
    x,
    hist_steps,
    gen_block,
    num_insertions,
    num_coolings,
    n_gen_msgs,
    midprice_step_size
)
fig.show()

insertion positions: [551, 602, 653, 704, 755, 806, 857, 908, 959, 1010]
cooling positions:   [1061, 1112, 1163, 1214, 1265, 1316, 1367, 1418, 1469, 1520, 1571, 1622, 1673, 1724, 1775, 1826, 1877, 1928, 1979, 2030, 2081, 2132, 2183, 2234, 2285, 2336, 2387, 2438, 2489, 2540]


In [8]:
# x, all_series, merged, hist_steps, gen_block = prepare_volatility_filtered_series(merged, hist_msgs, n_gen_msgs, midprice_step_size, volatility_cutoff=0.3)

In [9]:
fig, mean_series, std_series = plot_midprice_series_with_mean_std(
    merged=merged,
    all_series=all_series,
    x=x,
    hist_steps=hist_steps,
    gen_block=gen_block,
    num_insertions=num_insertions,
    num_coolings=num_coolings,
    n_gen_msgs=n_gen_msgs,
    midprice_step_size=midprice_step_size,
)
fig.show()

insertion positions: [551, 602, 653, 704, 755, 806, 857, 908, 959, 1010]
cooling positions:   [1061, 1112, 1163, 1214, 1265, 1316, 1367, 1418, 1469, 1520, 1571, 1622, 1673, 1724, 1775, 1826, 1877, 1928, 1979, 2030, 2081, 2132, 2183, 2234, 2285, 2336, 2387, 2438, 2489, 2540]


In [10]:
# np.save("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_mean.npy", mean_series)
# np.save("/app/data_saved/exp_96_20250703_212149/exp_96_20250703_212149_std.npy", std_series)

# ----------------------------------------------------

# ----------------------------------------------------

# ----------------------------------------------------

# OREDER-PLAYER

In [11]:
# import os, glob, re
# import numpy as np
# import pandas as pd

# # PRINT EACH NAME + SHAPE WHEN LOADING
# def build_and_merge(folder, batch_prefix, inp_prefix):
#     # STEP 1: load every .npy (shape (batch_size, time, feat)) into a DataFrame
#     files   = glob.glob(os.path.join(folder, "*.npy"))
#     rx_iter = re.compile(rf"{re.escape(batch_prefix)}_\[(.+)\]_iter_(\d+)\.npy$")
#     rx_inp  = re.compile(rf"{re.escape(inp_prefix)}_\[(.+)\]\.npy$")
#     rec = []
#     for f in files:
#         nm = os.path.basename(f)
#         m  = rx_iter.match(nm)
#         if m:
#             rng, itr = m.group(1).replace(" ", ""), int(m.group(2))
#         else:
#             m2 = rx_inp.match(nm)
#             if not m2:
#                 continue
#             rng, itr = m2.group(1).replace(" ", ""), 0
#         batch = np.load(f)  # shape (batch_size, time, features)

#         # ======== PRINT FILE NAME AND SHAPE =========
#         print(f"Loaded {nm} with shape {batch.shape}")
#         # ============================================

#         rec.append({"range": rng, "iteration": itr, "batch": batch})
#     df = pd.DataFrame(rec).sort_values(["range","iteration"]).reset_index(drop=True)

#     # STEP 2: parse the comma‐separated list of IDs into Python lists
#     df["ids"] = df["range"].str.split(",").apply(lambda L: [int(x) for x in L])

#     # explode each batch into one row per sample
#     rows = []
#     for _, r in df.iterrows():
#         for idx, sample_id in enumerate(r["ids"]):
#             single = r["batch"][idx]   # shape (time, features)
#             rows.append({
#                 "id":        sample_id,
#                 "iteration": r["iteration"],
#                 "data":      single
#             })
#     df_sorted = pd.DataFrame(rows).sort_values(["id","iteration"]).reset_index(drop=True)

#     # STEP 3: for each id, concatenate all its iterations end-to-end
#     merged = []
#     for id_val, grp in df_sorted.groupby("id", sort=True):
#         arrs = [row.data for _, row in grp.iterrows()]
#         big  = np.concatenate(arrs, axis=0)   # (sum_time, features)
#         merged.append({"id": id_val, "merged_data": big})
#     merged_df = pd.DataFrame(merged).sort_values("id").reset_index(drop=True)

#     return df, df_sorted, merged_df

# # — example usage —
# b_folder      = f"/app/data_saved/{experiment_name}/b_seq_gen_doubled"
# b_batch_pref  = "b_seq_gen_doubled_batch"
# b_inp_pref    = "b_seq_inp"
# _, b_sorted, b_merged = build_and_merge(b_folder, b_batch_pref, b_inp_pref)

# m_folder      = f"/app/data_saved/{experiment_name}/msgs_decoded_doubled"
# m_batch_pref  = "msgs_decoded_doubled_batch"
# m_inp_pref    = "m_seq_raw_inp"
# _, m_sorted, m_merged = build_and_merge(m_folder, m_batch_pref, m_inp_pref)

# # build your dicts
# b_dict = { int(r.id): np.array(r.merged_data) for _, r in b_merged.iterrows() }
# m_dict = { int(r.id): np.array(r.merged_data) for _, r in m_merged.iterrows() }

# # prepend zero‐row so that your interactive plot indexing from t=1…T works
# for d in (b_dict, m_dict):
#     for key, arr in d.items():
#         zero = np.zeros((1, arr.shape[1]), dtype=arr.dtype)
#         d[key] = np.vstack([zero, arr])

In [12]:
import os, glob, re
import numpy as np
import pandas as pd

def build_and_merge(folder, batch_prefix, inp_prefix):
    # STEP 1: load every .npy (shape (batch_size, time, feat)) into a DataFrame
    files   = glob.glob(os.path.join(folder, "*.npy"))
    rx_iter = re.compile(rf"{re.escape(batch_prefix)}_\[(.+)\]_iter_(\d+)\.npy$")
    rx_inp  = re.compile(rf"{re.escape(inp_prefix)}_\[(.+)\]\.npy$")
    rec = []
    for f in files:
        nm = os.path.basename(f)
        m  = rx_iter.match(nm)
        if m:
            rng, itr = m.group(1).replace(" ", ""), int(m.group(2))
        else:
            m2 = rx_inp.match(nm)
            if not m2:
                continue
            rng, itr = m2.group(1).replace(" ", ""), 0

        batch = np.load(f)  # shape (batch_size, time, features)
        print(f"Loaded {nm} with shape {batch.shape}")

        rec.append({"range": rng, "iteration": itr, "batch": batch})
    df = pd.DataFrame(rec).sort_values(["range","iteration"]).reset_index(drop=True)

    # STEP 2: parse the comma‐separated list of IDs into Python lists
    df["ids"] = df["range"].str.split(",").apply(lambda L: [int(x) for x in L])

    # explode each batch into one row per sample, с учётом slicing
    rows = []
    for _, r in df.iterrows():
        for idx, sample_id in enumerate(r["ids"]):
            single = r["batch"][idx]   # shape (time, features)

            # ====== здесь происходит нужный slice ======
            if r["iteration"] > 0:
                # для первой генерации — последние 51, для остальных — по 50
                n_keep = 51 if r["iteration"] == 1 else 50
                single = single[-n_keep:, :]
            # ============================================

            rows.append({
                "id":        sample_id,
                "iteration": r["iteration"],
                "data":      single
            })

    df_sorted = pd.DataFrame(rows).sort_values(["id","iteration"]).reset_index(drop=True)

    # STEP 3: для каждого id склеить его кусочки
    merged = []
    for id_val, grp in df_sorted.groupby("id", sort=True):
        arrs = [row.data for _, row in grp.iterrows()]
        big  = np.concatenate(arrs, axis=0)   # (sum_time, features)
        merged.append({"id": id_val, "merged_data": big})
    merged_df = pd.DataFrame(merged).sort_values("id").reset_index(drop=True)

    return df, df_sorted, merged_df

# b_folder      = f"/app/data_saved/{experiment_name}/l2_book_states_halved"
# b_batch_pref  = "l2_book_states_halved_batch" # "b_seq_gen_doubled_batch" 
# b_inp_pref    = "l2_book_states_halved_batch"

b_folder      = f"/app/data_saved/{experiment_name}/b_seq_gen_doubled"
b_batch_pref  = "b_seq_gen_doubled_batch"
b_inp_pref    = "b_seq_inp"

m_folder      = f"/app/data_saved/{experiment_name}/msgs_decoded_doubled"
m_batch_pref  = "msgs_decoded_doubled_batch"
m_inp_pref    = "m_seq_raw_inp"

_, b_sorted, b_merged = build_and_merge(b_folder, b_batch_pref, b_inp_pref)
_, m_sorted, m_merged = build_and_merge(m_folder, m_batch_pref, m_inp_pref)

# Словари
b_dict = { int(r.id): np.array(r.merged_data) for _, r in b_merged.iterrows() }
m_dict = { int(r.id): np.array(r.merged_data) for _, r in m_merged.iterrows() }

# prepend zero‐row для интерактивного индекса
for d in (b_dict, m_dict):
    for key, arr in d.items():
        zero = np.zeros((1, arr.shape[1]), dtype=arr.dtype)
        d[key] = np.vstack([zero, arr])

Loaded b_seq_gen_doubled_batch_[2640, 4030, 1086, 2674]_iter_0.npy with shape (4, 2510, 501)
Loaded msgs_decoded_doubled_batch_[2640, 4030, 1086, 2674]_iter_0.npy with shape (4, 2510, 14)


In [13]:
# m_dict[14606].shape

In [14]:
# b_dict[14606].shape

In [15]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

def interactive_lob_plot(b_seq_inp, msg_seq_raw):
    # allow DataFrame or dict
    if isinstance(b_seq_inp, pd.DataFrame):
        b_seq_inp = {int(r.id): np.array(r.merged_data) for _,r in b_seq_inp.iterrows()}
    if isinstance(msg_seq_raw, pd.DataFrame):
        msg_seq_raw = {int(r.id): np.array(r.merged_data) for _,r in msg_seq_raw.iterrows()}

    # controls
    id_dd       = widgets.Dropdown(options=sorted(b_seq_inp.keys()), description="Sample ID:")
    time_slider = widgets.IntSlider(min=1, max=1, step=1, description="t:")
    btn_prev    = widgets.Button(description="←")
    btn_next    = widgets.Button(description="→")
    msg_box     = widgets.HTML()

    # figure with two subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Book state t–1","Book state t"])
    fig.add_trace(go.Bar(x=[],y=[]), row=1,col=1)
    fig.add_trace(go.Bar(x=[],y=[]), row=1,col=2)
    fig.update_layout(width=800, height=400, showlegend=False, template='plotly_white')
    fig_widget = go.FigureWidget(fig)

    def update_slider_range(*_):
        arr = b_seq_inp[id_dd.value]
        # now slider runs 1…(T−1)
        time_slider.min = 1
        time_slider.max = arr.shape[0] - 1
        time_slider.value = 1

    def update_plot(*_):
        sid = id_dd.value
        t   = time_slider.value
        arr = b_seq_inp[sid]
        msgs= msg_seq_raw[sid]

        # take book states at t−1 and t
        s0 = arr[t-1, 240:263]
        s1 = arr[t,   240:263]

        # s0 = arr[t-1]
        # s1 = arr[t]


        diff = abs(s1) - abs(s0)
        x = np.arange(len(s0)) - len(s0)//2

        with fig_widget.batch_update():
            fig_widget.data = []
            fig_widget.add_bar(x=x, y=s0, row=1, col=1, marker_color='orange')
            
            colors = ['orange' if abs(d)<1e-8 else ('red' if d>0 else 'blue') for d in diff]
                        
            fig_widget.add_bar(x=x, y=s1, row=1, col=2, marker_color=colors)
            fig_widget.layout.annotations[0].text = f"Book state {t-1}"
            fig_widget.layout.annotations[1].text = f"Book state {t}"

        # show message at index t
        m = msgs[t].astype(int)
        # fields: [0]=timestamp, [1]=etype, [2]=dir, [3]=abspr, [4]=relpr, [5]=size, …
        et, dr, abspr, relpr, sz = m[1], m[2], m[3], m[4], m[5]
        et_map = {1:"Limit",2:"PartialCancel",3:"Delete",4:"Execution"}
        dr_map = {1:"Buy",0:"Sell"}
        info = (
            f"{et_map.get(et,'?')} • "
            f"{dr_map.get(dr,'?')} • "
            f"abs={abspr} • rel={relpr} • size={sz}"
        )
        msg_box.value = f"<b>{info}<br></b>raw:{m.tolist()}"

    def on_prev(b):
        if time_slider.value>time_slider.min:
            time_slider.value -= 1
    def on_next(b):
        if time_slider.value<time_slider.max:
            time_slider.value += 1

    # wire up events
    id_dd.observe(lambda c: update_slider_range(), names='value')
    time_slider.observe(lambda c: update_plot(), names='value')
    btn_prev.on_click(on_prev)
    btn_next.on_click(on_next)

    # initial draw
    update_slider_range()
    update_plot()

    display(widgets.HBox([id_dd, btn_prev, btn_next, time_slider]))
    display(fig_widget, msg_box)


interactive_lob_plot(b_dict, m_dict)

FigureWidget({
    'data': [{'marker': {'color': 'orange'},
              'type': 'bar',
              'uid': 'd5bfb0a0-649d-4091-adeb-325e35030325',
              'x': {'bdata': '9fb3+Pn6+/z9/v8AAQIDBAUGBwgJCgs=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f4'},
              'yaxis': 'y'},
             {'marker': {'color': [orange, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, orange, orange]},
              'type': 'bar',
              'uid': '538bb4dc-7ba2-4c4b-9e84-cd7756a2b5d5',
              'x': {'bdata': '9fb3+Pn6+/z9/v8AAQIDBAUGBwgJCgs=', 'dtype': 'i1'},
              'xaxis': 'x2',
              'y': {'bdata': ('AAAAADQzMz4fhQs/zcwsP8l2/j6EwE' ... 'G/rkcBv2dm5r7gT82+AAAAAAAAAAA='),
                    'dtype': 'f4'},
     

HTML(value='<b>Delete • Sell • abs=882100 • rel=3 • size=100<br></b>raw:[209410446, 3, 0, 882100, 3, 100, 0, 6…

In [16]:
def show_execution_indices(m_dict):
    """
    For each sample_id in m_dict, prints indices t where etype == 4 (Execution).
    """
    for sid, msgs in m_dict.items():
        etypes = msgs[:, 1]
        exec_indices = np.where(etypes == 4)[0]
        if len(exec_indices) > 0:
            print(f"Sample ID {sid}: Execution at indices {exec_indices.tolist()}")
        else:
            print(f"Sample ID {sid}: No Executions found")

show_execution_indices(m_dict)

Sample ID 1086: Execution at indices [15, 213, 268, 683, 1020, 1026, 1027, 1482, 1483, 1484, 1485, 1486, 1514, 1627, 1629, 1630, 1679, 1680, 1701, 1702, 1739, 1740, 1932, 1981, 2135, 2229]
Sample ID 2640: Execution at indices [50, 87, 229, 306, 307, 324, 325, 333, 344, 389, 494, 504, 505, 506, 515, 529, 530, 533, 534, 535, 552, 591, 634, 855, 858, 979, 980, 1091, 1220, 1221, 1303, 1311, 1446, 1447, 1464, 1525, 1555, 1604, 1762, 1763, 1844, 1846, 1847, 1853, 1919, 1981, 1982, 2149, 2150, 2155, 2156]
Sample ID 2674: Execution at indices [62, 230, 585, 586, 587, 739, 876, 1219, 1220, 1720, 1721, 1876, 1878, 1879, 1880, 1881, 2240, 2435, 2436]
Sample ID 4030: Execution at indices [6, 7, 8, 9, 10, 28, 181, 216, 244, 245, 282, 283, 328, 356, 500, 510, 511, 512, 513, 525, 662, 785, 793, 899, 951, 1089, 1102, 1103, 1312, 1365, 1635, 1636, 1920, 1976, 2037, 2038, 2167, 2392]
